In [ ]:
#| default_exp utils

# Aggregation/Visualization Utils

The `HierarchicalForecast` package contains utility functions to wrangle and visualize 
hierarchical series datasets. The `aggregate` function of the module allows you to create
a hierarchy from categorical variables representing the structure levels, returning also
the aggregation contraints matrix $\mathbf{S}$.

In addition, `HierarchicalForecast` ensures compatibility of its reconciliation methods with other popular machine-learning libraries via its external forecast adapters that transform output base forecasts from external libraries into a compatible data frame format.

In [ ]:
#| export
import itertools
import reprlib
import sys
import timeit

import matplotlib.pyplot as plt
import narwhals as nw
import numpy as np
import pandas as pd

from narwhals.typing import Frame, FrameT
from numba import njit, prange
from sklearn.preprocessing import OneHotEncoder
from typing import Optional, Union, Sequence
import utilsforecast.feature_engineering as ufe
import utilsforecast.processing as ufp

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from fastcore.test import test_eq, test_close, test_fail
from statsforecast.utils import generate_series

In [ ]:
#| exporti
# Global variables
NUMBA_NOGIL = True
NUMBA_CACHE = True
NUMBA_PARALLEL = True
NUMBA_FASTMATH = True

In [ ]:
#| exporti
class CodeTimer:
    def __init__(self, name=None, verbose=True):
        self.name = " '"  + name + "'" if name else ''
        self.verbose = verbose

    def __enter__(self):
        self.start = timeit.default_timer()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (timeit.default_timer() - self.start)
        if self.verbose:
            print('Code block' + self.name + \
                  ' took:\t{0:.5f}'.format(self.took) + ' seconds')

In [ ]:
#| exporti
def is_strictly_hierarchical(S: np.ndarray, 
                             tags: dict[str, np.ndarray]) -> bool:
    # main idea:
    # if S represents a strictly hierarchical structure
    # the number of paths before the bottom level
    # should be equal to the number of nodes
    # of the previuos level
    levels_ = dict(sorted(tags.items(), key=lambda x: len(x[1])))
    # removing bottom level
    levels_.popitem()
    # making S categorical
    hiers = [np.argmax(S[idx], axis=0) + 1 for _, idx in levels_.items()]
    hiers = np.vstack(hiers)
    paths = np.unique(hiers, axis=1).shape[1] 
    nodes = levels_.popitem()[1].size
    return paths == nodes

# Aggregate Function

In [ ]:
#| exporti
def _to_upper_hierarchy(bottom_split: list[str], bottom_values: str, upper_key: str) -> list[str]:
    upper_split = upper_key.split('/')
    upper_idxs = [bottom_split.index(i) for i in upper_split]

    def join_upper(bottom_value):
        bottom_parts = bottom_value.split('/')
        return '/'.join(bottom_parts[i] for i in upper_idxs)

    return [join_upper(val) for val in bottom_values]

In [ ]:
#| export
def aggregate(
    df: Frame,
    spec: list[list[str]],
    exog_vars: Optional[dict[str, Union[str, list[str]]]] = None,
    sparse_s: bool = False,
    id_col: str = "unique_id",
    time_col: str = "ds", 
    target_cols: list[str] = ["y"],      
) -> tuple[FrameT, FrameT, dict]:
    """Utils Aggregation Function.
    Aggregates bottom level series contained in the DataFrame `df` according
    to levels defined in the `spec` list.

    Parameters
    ----------
    df : DataFrame
        Dataframe with columns `[time_col, *target_cols]`, columns to aggregate and optionally exog_vars.
    spec : list of list of str
        list of levels. Each element of the list should contain a list of columns of `df` to aggregate.
    exog_vars: dictionary of string keys & values that can either be a list of strings or a single string
        keys correspond to column names and the values represent the aggregation(s) that will be applied to each column. Accepted values are those from Pandas or Polars aggregation Functions, check the respective docs for guidance
    is_balanced : bool (default=False)
        Deprecated.        
    sparse_s : bool (default=False)
        Return `S_df` as a sparse Pandas dataframe.
    id_col : str (default='unique_id')
        Column that will identify each serie after aggregation.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.
    target_cols : (default=['y'])
        list of columns that contains the targets to aggregate.        

    Returns
    -------
    Y_df : DataFrame
        Hierarchically structured series.
    S_df : DataFrame
        Summing dataframe.
    tags : dict
        Aggregation indices.
    """
    # To Narwhals
    df_nw = nw.from_native(df)
    native_namespace = nw.get_native_namespace(df_nw)

    # Checks
    # Generate order-preserving list of unique cols based on spec
    seen = set()
    spec_cols = [col for cols in spec for col in cols if col not in seen and not seen.add(col)]  # type: ignore[func-returns-value]
    
    # Check if last level in spec contains all levels
    missing_cols_in_bottom_spec = set(spec_cols) - set(spec[-1])
    if len(missing_cols_in_bottom_spec) > 0:
        raise ValueError(f"Check the last (bottom) level of spec, it has missing columns: {reprlib.repr(missing_cols_in_bottom_spec)}")

    if sparse_s and not nw.dependencies.is_pandas_dataframe(df):
        raise ValueError("Sparse output is only supported for Pandas DataFrames.")
    
    for col in df_nw.columns:
        if df_nw[col].is_null().any():
            raise ValueError(f"Column {col} contains null values. Make sure no column in the DataFrame contains null values.")

    # Check whether all columns in the spec are in the df
    aggregation_cols_in_spec = list(dict.fromkeys([col for cols in spec for col in cols]))
    for col in aggregation_cols_in_spec:
        if col not in df_nw.columns:
            raise ValueError(f"Column {col} in spec not present in df")

    # Prepare the aggregation dictionary  
    agg_dict = dict(zip(target_cols, tuple(zip(target_cols, len(target_cols)*["sum"]))))

    # Check if exog_vars are present in df & add to the aggregation dictionary if it is not None
    exog_var_names = []
    if exog_vars is not None:
        missing_vars = [var for var in exog_vars.keys() if var not in df.columns]
        if missing_vars:
            raise ValueError(f"The following exogenous variables are not present in the DataFrame: {', '.join(missing_vars)}")    
        else:
          # Update agg_dict to handle multiple aggregations for each exog_vars key
            for key, agg_func in exog_vars.items():
                # Ensure agg_func is a list
                if isinstance(agg_func, str):  # If it's a single string, convert to list
                    agg_func = [agg_func]
                elif not isinstance(agg_func, list):  # Raise an error if it's neither
                    raise ValueError(f"Aggregation functions for '{key}' must be a string or a list of strings.")
                
                for func in agg_func:
                    agg_dict[f"{key}_{func}"] = (key, func)  # Update the agg_dict with the new naming structure
                    exog_var_names.append(f"{key}_{func}")

    # compute aggregations and tags
    spec = sorted(spec, key=len)

    tags = {}
    Y_nws = []
    category_list = []
    level_sep = "/"
    # Perform the aggregation
    for level in spec:
        level_name = '/'.join(level)

        # Create Y_df
        Y_level = df_nw.group_by(level + [time_col]).agg(
            *[getattr(nw.col(col), agg)().alias(col_name) for col_name, (col, agg) in agg_dict.items()]
        )
        Y_level = Y_level.select(nw.concat_str( [nw.col(col) for col in level],
                separator=level_sep).alias(id_col), 
                            nw.all())
        Y_level = Y_level[[id_col, time_col, *target_cols] + exog_var_names]
        Y_level = Y_level.sort(by=[id_col, time_col])
        Y_nws.append(Y_level)

        tags[level_name] = Y_level[id_col].unique().sort().to_numpy()
        category_list.extend(tags[level_name])

    Y_nw = nw.concat(Y_nws, how="vertical")
    Y_nw = nw.maybe_reset_index(Y_nw)
    Y_df = Y_nw.to_native()
    
    # construct S
    bottom = spec[-1]
    bottom_levels = tags[level_name]
    S = np.empty((len(bottom_levels), len(spec)), dtype=object)

    for j, levels in enumerate(spec[:-1]):
        S[:, j] = _to_upper_hierarchy(bottom, bottom_levels, '/'.join(levels))
    S[:, -1] = tags[level_name]
    categories = list(tags.values())
    
    encoder = OneHotEncoder(categories=categories, sparse_output=sparse_s, dtype=np.float64)  
    S_dum = encoder.fit_transform(S)
    
    if not sparse_s:
        S_nw = nw.from_dict({
                            **{id_col: category_list},
                            **dict(zip(tags[level_name], S_dum)),
                            }, 
                            native_namespace=native_namespace)
        S_nw = nw.maybe_reset_index(S_nw)
        S_df = S_nw.to_native()
    else:
        S_df = pd.DataFrame.sparse.from_spmatrix(S_dum.T, columns=list(bottom_levels),
                                                 index=category_list)
        S_df = S_df.reset_index(names=id_col)

    return Y_df, S_df, tags

In [ ]:
show_doc(aggregate, title_level=3)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/utils.py#L81){target="_blank" style="float:right; font-size:smaller"}

### aggregate

>      aggregate
>                 (df:Union[ForwardRef('DataFrame[Any]'),ForwardRef('LazyFrame[A
>                 ny]')], spec:list[list[str]],
>                 exog_vars:Optional[dict[str,Union[str,list[str]]]]=None,
>                 sparse_s:bool=False, id_col:str='unique_id',
>                 time_col:str='ds', target_cols:list[str]=['y'])

*Utils Aggregation Function.
Aggregates bottom level series contained in the DataFrame `df` according
to levels defined in the `spec` list.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Dataframe with columns `[time_col, *target_cols]`, columns to aggregate and optionally exog_vars. |
| spec | list |  | list of levels. Each element of the list should contain a list of columns of `df` to aggregate. |
| exog_vars | Optional | None |  |
| sparse_s | bool | False | Return `S_df` as a sparse Pandas dataframe. |
| id_col | str | unique_id | Column that will identify each serie after aggregation. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_cols | list | ['y'] | list of columns that contains the targets to aggregate.         |
| **Returns** | **tuple** |  | **Hierarchically structured series.** |

In [ ]:
#| hide
# simple case
df = pd.DataFrame(
    {
        'cat1': ['a', 'a', 'a', 'b'],
        'cat2': ['1', '2', '3', '2'],
        'y': [10, 20, 30, 40],
        'ds': ['2020-01-01', '2020-02-01', '2020-03-01', '2020-02-01']
    }
)
df['country'] = 'COUNTRY'
spec = [['country'], ['country', 'cat1'], ['country', 'cat1', 'cat2']]
Y_df, S_df, tags = aggregate(df, spec)
test_eq(
    list(Y_df["unique_id"]), 
    3 * ['COUNTRY'] +
    3 * ['COUNTRY/a'] +
    ['COUNTRY/b'] +
    ['COUNTRY/a/1', 'COUNTRY/a/2', 'COUNTRY/a/3'] +
    ['COUNTRY/b/2']
)
test_eq(Y_df.query("unique_id == 'COUNTRY'")['y'].values, [10, 60, 30])
test_eq(
    list(S_df["unique_id"]),
    ['COUNTRY', 'COUNTRY/a', 'COUNTRY/b', 'COUNTRY/a/1', 'COUNTRY/a/2', 'COUNTRY/a/3', 'COUNTRY/b/2'],
)
test_eq(
    S_df.columns,
    ['unique_id', 'COUNTRY/a/1', 'COUNTRY/a/2', 'COUNTRY/a/3', 'COUNTRY/b/2'],
)
expected_tags = {
    'country': ['COUNTRY'],
    'country/cat1': ['COUNTRY/a', 'COUNTRY/b'],
    'country/cat1/cat2': ['COUNTRY/a/1', 'COUNTRY/a/2', 'COUNTRY/a/3','COUNTRY/b/2'],
}
for k, actual in tags.items():
    test_eq(actual, expected_tags[k])

# test categoricals don't produce all combinations
df2 = df.copy()
for col in ('country', 'cat1', 'cat2'):
    df2[col] = df2[col].astype('category')

Y_df2, *_ = aggregate(df2, spec)
assert Y_df.shape[0] == Y_df2.shape[0]

In [ ]:
#| hide
import polars as pl

In [ ]:
#| hide
# Testing equivalence of other dataframe libs to pandas results. 
# TODO: extend for other frameworks
def test_eq_agg_dataframe(df, spec, frameworks=["polars"]):
    for framework in frameworks:
        if framework == "polars":
            df_f = pl.from_pandas(df)
        else:
            raise ValueError(f"Framework {framework} not recognized")
        
        Y_df, S_df, tags = aggregate(df=df, spec=spec)
        Y_df_f, S_df_f, tags_f = aggregate(df=df_f, spec=spec)

        pd.testing.assert_frame_equal(Y_df, Y_df_f.to_pandas())
        pd.testing.assert_frame_equal(S_df, S_df_f.to_pandas())
        for tag in tags:
            test_eq(tags[tag], tags_f[tag])    

In [ ]:
#| hide
# polars
test_eq_agg_dataframe(df, spec)

In [ ]:
#| hide
# test unbalanced dataset
max_tenure = 24
dates = pd.date_range(start='2019-01-31', freq='M', periods=max_tenure)
cohort_tenure = [24, 23, 22, 21]

ts_list = []

# Create ts for each cohort
for i in range(len(cohort_tenure)):
    ts_list.append(
        generate_series(n_series=1, freq='M', min_length=cohort_tenure[i], max_length=cohort_tenure[i]).reset_index() \
            .assign(ult=i) \
            .assign(ds=dates[-cohort_tenure[i]:]) \
            .drop(columns=['unique_id'])
    )
df = pd.concat(ts_list, ignore_index=True)

# Create categories
df['pen'] = np.where(df['ult'] < 2, 'a', 'b')
# Note that unique id requires strings
df['ult'] = df['ult'].astype(str)

hier_levels = [
    ['pen'],
    ['pen', 'ult'],
]

hier_df, S_df, tags = aggregate(df=df, spec=hier_levels)

C:\Users\ospra\AppData\Local\Temp\ipykernel_7984\2654088726.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2019-01-31', freq='M', periods=max_tenure)
c:\Users\ospra\miniconda3\envs\hierarchicalforecast-backup\lib\site-packages\utilsforecast\data.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\ospra\miniconda3\envs\hierarchicalforecast-backup\lib\site-packages\utilsforecast\data.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\ospra\miniconda3\envs\hierarchicalforecast-backup\lib\site-packages\utilsforecast\data.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users

In [ ]:
#| hide
# polars
test_eq_agg_dataframe(df, hier_levels)

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# strictly hierarchical structure
hiers_strictly = [['Country'],
                  ['Country', 'State'], 
                  ['Country', 'State', 'Region']]

# test strict
hier_df, S_df, tags = aggregate(df=df, spec=hiers_strictly)
test_eq(len(hier_df), 6800)
test_eq(hier_df["unique_id"].nunique(), 85)
test_eq(S_df.shape, (85, 77))
test_eq(hier_df["unique_id"].unique(), S_df["unique_id"])
test_eq(len(tags), len(hiers_strictly))                

# grouped structure
hiers_grouped = [['Country'],
                 ['Country', 'State'], 
                 ['Country', 'Purpose'], 
                 ['Country', 'State', 'Region'], 
                 ['Country', 'State', 'Purpose'], 
                 ['Country', 'State', 'Region', 'Purpose'],
                 ]

# test grouped
hier_df, S_df, tags = aggregate(df=df, spec=hiers_grouped)
test_eq(len(hier_df), 34_000)
test_eq(hier_df["unique_id"].nunique(), 425)
test_eq(S_df.shape, (425, 305))
test_eq(hier_df["unique_id"].unique(), S_df["unique_id"])
test_eq(len(tags), len(hiers_grouped))

In [ ]:
#| hide
# polars
test_eq_agg_dataframe(df, hiers_strictly)
test_eq_agg_dataframe(df, hiers_grouped)

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

#Unit Test NaN Values
df_nan = df.copy()
df_nan.loc[0, 'Region'] = float('nan')
test_fail(
    aggregate,
    contains='null values',
    args=(df_nan, hiers_strictly),
)

#Unit Test None Values
df_none = df.copy()
df_none.loc[0, 'Region'] = None
test_fail(
    aggregate,
    contains='null values',
    args=(df_none, hiers_strictly),
)

In [ ]:
#| hide
# polars

#Unit Test NaN Values
df_nan_pl = pl.DataFrame(df_nan)
test_fail(
    aggregate,
    contains='null values',
    args=(df_nan_pl, hiers_strictly),
)

#Unit Test None Values
df_none_pl = pl.DataFrame(df_none)
test_fail(
    aggregate,
    contains='null values',
    args=(df_none_pl, hiers_strictly),
)

In [ ]:
#| hide
# Test equality of sparse and non-sparse aggregation
with CodeTimer('strict non-sparse aggregate'):
    Y_df, S_df, tags = aggregate(df=df, sparse_s=False, spec=hiers_strictly)

with CodeTimer('strict sparse aggregate'):
    Y_df_sparse, S_df_sparse, tags_sparse = aggregate(df=df, sparse_s=True, spec=hiers_strictly)

test_close(Y_df.y.values, Y_df_sparse.y.values)
test_eq(S_df.values, S_df_sparse.values)

test_eq(S_df.columns, S_df_sparse.columns)
test_eq(S_df.index, S_df_sparse.index)

test_eq(Y_df.columns, Y_df_sparse.columns)
test_eq(Y_df.index, Y_df_sparse.index)

with CodeTimer('grouped non-sparse aggregate'):
    Y_df, S_df, tags = aggregate(df=df, sparse_s=False, spec=hiers_grouped)

with CodeTimer('grouped sparse aggregate'):
    Y_df_sparse, S_df_sparse, tags_sparse = aggregate(df=df, sparse_s=True, spec=hiers_grouped)

test_close(Y_df.y.values, Y_df_sparse.y.values)
test_eq(S_df.values, S_df_sparse.values)

test_eq(S_df.columns, S_df_sparse.columns)
test_eq(S_df.index, S_df_sparse.index)

test_eq(Y_df.columns, Y_df_sparse.columns)
test_eq(Y_df.index, Y_df_sparse.index)

Code block 'strict non-sparse aggregate' took:	0.03771 seconds
Code block 'strict sparse aggregate' took:	0.04140 seconds
Code block 'grouped non-sparse aggregate' took:	0.09591 seconds
Code block 'grouped sparse aggregate' took:	0.09406 seconds


In [ ]:
#| export
def aggregate_temporal(
    df: Frame,
    spec: list[list[str]],
    freq: Union[str, int],
    exog_vars: Optional[dict[str, Union[str, list[str]]]] = None,
    sparse_s: bool = False,
    id_col: str = "unique_id",
    time_col: str = "ds", 
    id_time_col: str = "temporal_id",
    target_cols: list[str] = ["y"],      
) -> tuple[FrameT, FrameT, dict]:
    """Utils Aggregation Function for Temporal aggregations.
    Aggregates bottom level timesteps contained in the DataFrame `df` according
    to temporal levels defined in the `spec` list.

    Parameters
    ----------
    df : DataFrame
        Dataframe with columns `[time_col, target_cols]` and columns to aggregate.
    spec : list of list of str
        List of temporal levels. May only contain string aliases of timestamp attributes.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    exog_vars: dictionary of string keys & values that can either be a list of strings or a single string
        keys correspond to column names and the values represent the aggregation(s) that will be applied to each column. Accepted values are those from Pandas or Polars aggregation Functions, check the respective docs for guidance
    sparse_s : bool (default=False)
        Return `S_df` as a sparse Pandas dataframe.
    id_col : str (default='unique_id')
        Column that will identify each serie after aggregation.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.
    id_time_col : str (default='temporal_id')
        Column that will identify each timestep after aggregation.
    target_cols : (default=['y'])
        List of columns that contain the targets to aggregate.        

    Returns
    -------
    Y_df : DataFrame
        Temporally hierarchically structured series.
    S_df : DataFrame
        Temporal summing dataframe.
    tags : dict
        Temporal aggregation indices.
    """
    # Generate order-preserving list of unique time features based on spec
    fseen = set()
    time_features = [col for cols in spec for col in cols if col not in fseen and not fseen.add(col)] # type: ignore[func-returns-value]
    # Check if last level in spec is equivalent to time features
    missing_time_features_in_bottom_spec = set(time_features) - set(spec[-1])
    if len(missing_time_features_in_bottom_spec) > 0:
        raise ValueError(f"Check the last (bottom) level of spec, it has missing time features: {reprlib.repr(missing_time_features_in_bottom_spec)}")
    
    # If target_cols is not in df, we add a placeholder column so that we can compute the aggregations
    add_placeholder = False
    if set(df.columns) == set([time_col, id_col]):
        add_placeholder = True
        df_nw = nw.from_native(df)
        df_nw = df_nw.with_columns(nw.lit(0).alias("y"))
        df = df_nw.to_native()

    # Compute time features
    time_features.remove(time_col)
    df, _ = ufe.time_features(df=df, 
                                freq=freq, 
                                h=0, 
                                features=time_features, 
                                time_col=time_col,
                                id_col=id_col)
   
    # Create the aggregation
    Y_df, S_df, tags = aggregate(df = df, 
                                spec = spec, 
                                exog_vars=exog_vars,
                                sparse_s=sparse_s,
                                id_col=id_time_col, 
                                target_cols=target_cols,
                                time_col=id_col)   
    Y_nw = nw.from_native(Y_df)

    # Add unique_id_ds column to Y_df
    # This is a bit dirty but Narwhals does not support string splitting
    timestamps = pd.to_datetime(Y_nw[id_time_col].to_pandas().str.split("/").str[-1], format="mixed").values
    Y_nw = Y_nw.with_columns(**{time_col: timestamps})

    # Drop the placeholder column if it was added
    if add_placeholder:
        Y_nw = Y_nw.drop("y")

    Y_df = Y_nw.to_native()

    return Y_df, S_df, tags

In [ ]:
show_doc(aggregate_temporal, title_level=3)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/utils.py#L257){target="_blank" style="float:right; font-size:smaller"}

### aggregate_temporal

>      aggregate_temporal
>                          (df:Union[ForwardRef('DataFrame[Any]'),ForwardRef('La
>                          zyFrame[Any]')], spec:list[list[str]],
>                          freq:Union[str,int], exog_vars:Optional[dict[str,Unio
>                          n[str,list[str]]]]=None, sparse_s:bool=False,
>                          id_col:str='unique_id', time_col:str='ds',
>                          id_time_col:str='temporal_id',
>                          target_cols:list[str]=['y'])

*Utils Aggregation Function for Temporal aggregations.
Aggregates bottom level timesteps contained in the DataFrame `df` according
to temporal levels defined in the `spec` list.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Dataframe with columns `[time_col, target_cols]` and columns to aggregate. |
| spec | list |  | List of temporal levels. May only contain string aliases of timestamp attributes. |
| freq | Union |  | Frequency of the data. Must be a valid pandas or polars offset alias, or an integer. |
| exog_vars | Optional | None |  |
| sparse_s | bool | False | Return `S_df` as a sparse Pandas dataframe. |
| id_col | str | unique_id | Column that will identify each serie after aggregation. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| id_time_col | str | temporal_id | Column that will identify each timestep after aggregation. |
| target_cols | list | ['y'] | List of columns that contain the targets to aggregate.         |
| **Returns** | **tuple** |  | **Temporally hierarchically structured series.** |

In [ ]:
#| hide
# simple case
df = pd.DataFrame(
    {
        'cat1': 8 * ['a'] + 8 * ['b'],
        'y': np.linspace(10, 160, 16),
        'ds': 2 * ['2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01']
    }
)
df['country'] = 'COUNTRY'
df["unique_id"] = df["country"] + "/" + df["cat1"]
df["ds"] = pd.to_datetime(df["ds"])

spec = [['year'], ['year', 'ds']]
Y_df, S_df, tags = aggregate_temporal(df, spec, freq='QS')

test_eq(
    list(Y_df["unique_id"]), 
    10 * ['COUNTRY/a', 'COUNTRY/b'] 
)
test_eq(
    list(Y_df["temporal_id"]), 
    2 * ['2020'] + 2 * ['2021'] + 2 * ['2020/2020-01-01'] + 2 * ['2020/2020-04-01'] + 2 * ['2020/2020-07-01'] + 2 * ['2020/2020-10-01'] + 2 * ['2021/2021-01-01'] + 2 * ['2021/2021-04-01'] + 2 * ['2021/2021-07-01'] + 2 * ['2021/2021-10-01']
)
test_eq(Y_df.query("unique_id == 'COUNTRY/a' & temporal_id in ['2020', '2021']")['y'].values, [100, 260])
test_eq(
    list(S_df["temporal_id"]),
    ['2020', '2021', '2020/2020-01-01', '2020/2020-04-01', '2020/2020-07-01', '2020/2020-10-01', '2021/2021-01-01', '2021/2021-04-01', '2021/2021-07-01', '2021/2021-10-01'],
)
test_eq(
    S_df.columns,
    ['temporal_id', '2020/2020-01-01', '2020/2020-04-01', '2020/2020-07-01', '2020/2020-10-01', '2021/2021-01-01', '2021/2021-04-01', '2021/2021-07-01', '2021/2021-10-01'],
)
expected_tags = {
    'year': ['2020', '2021'],
    'year/ds': ['2020/2020-01-01', '2020/2020-04-01', '2020/2020-07-01', '2020/2020-10-01', '2021/2021-01-01', '2021/2021-04-01', '2021/2021-07-01', '2021/2021-10-01'],
}
for k, actual in tags.items():
    test_eq(actual, expected_tags[k])

In [ ]:
#| hide
# Testing equivalence of other dataframe libs to pandas results. 
# TODO: extend for other frameworks
def test_eq_agg_dataframe(df, spec, frameworks=["polars"]):
    for framework in frameworks:
        if framework == "polars":
            df_f = pl.from_pandas(df)
        else:
            raise ValueError(f"Framework {framework} not recognized")
        
        Y_df, S_df, tags = aggregate_temporal(df=df, spec=spec, freq='QS')
        Y_df_f, S_df_f, tags_f = aggregate_temporal(df=df_f, spec=spec, freq='QS')

        pd.testing.assert_frame_equal(Y_df[["unique_id", "ds", "y"]], Y_df_f[["unique_id", "ds", "y"]].to_pandas())
        np.testing.assert_array_equal(S_df.iloc[1:, 1:], S_df_f.to_pandas().iloc[1:, 1:])
        # for tag in tags:
        #     test_eq(tags[tag], tags_f[tag])  

In [ ]:
#| hide
# polars
test_eq_agg_dataframe(df, spec)

In [ ]:
#| export
def make_future_dataframe(
    df: Frame,
    freq: Union[str, int],
    h: int,
    id_col: str = "unique_id",
    time_col: str = "ds",
) -> FrameT:
    """Create future dataframe for forecasting.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Dataframe with ids, times and values for the exogenous regressors.
    freq : str or int
        Frequency of the data. Must be a valid pandas or polars offset alias, or an integer.
    h : int
        Forecast horizon.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.

    Returns
    -------
    future_df : pandas or polars DataFrame
        DataFrame with future values
    """
    times_by_id = ufp.group_by_agg(df, id_col, {time_col: "max"}, maintain_order=True)
    times_by_id = ufp.sort(times_by_id, id_col)
    future_df = ufp.make_future_dataframe(
        uids=times_by_id[id_col],
        last_times=times_by_id[time_col],
        freq=freq,
        h=h,
        id_col=id_col,
        time_col=time_col,
    )
    return future_df

In [ ]:
show_doc(make_future_dataframe, title_level=3)

In [ ]:
#| export
def get_cross_temporal_tags(df: Frame, 
                            tags_cs: dict[str, np.ndarray], 
                            tags_te: dict[str, np.ndarray], 
                            sep: str = "//", 
                            id_col: str = "unique_id", 
                            id_time_col: str = "temporal_id", 
                            cross_temporal_id_col: str = "cross_temporal_id") -> tuple[FrameT, dict[str, np.ndarray]]:
    """Get cross-temporal tags.

    Parameters
    ----------
    df : DataFrame
        DataFrame with temporal ids.
    tags_cs : dict
        Tags for the cross-sectional hierarchies
    tags_te : dict
        Tags for the temporal hierarchies
    sep : str (default="//")
        Separator for the cross-temporal tags.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    id_time_col : str (default='temporal_id')
        Column that identifies each (aggregated) timestep.
    cross_temporal_id_col : str (default='cross_temporal_id')
        Column that will identify each cross-temporal aggregation.

    Returns
    -------
    df: DataFrame
        DataFrame with cross-temporal ids.
    tags_ct : dict
        Tags for the cross-temporal hierarchies
    """
    tags_ct = {}
    for key_cs, value_cs in tags_cs.items():
        for key_te, value_te in tags_te.items():
            key_ct = key_cs + "//" + key_te
            value_ct = list("//".join(s) for s in itertools.product(value_cs, value_te))
            tags_ct[key_ct] = value_ct

    df_nw = nw.from_native(df)
    df_nw = df_nw.with_columns(**{cross_temporal_id_col: df_nw[id_col] + sep + df_nw[id_time_col]})
    df = df_nw.to_native()

    return df, tags_ct

In [ ]:
show_doc(get_cross_temporal_tags, title_level=3)

# Hierarchical Visualization

In [ ]:
#| export
class HierarchicalPlot:
    """ Hierarchical Plot

    This class contains a collection of matplotlib visualization methods, suited for small
    to medium sized hierarchical series.

    **Parameters:**<br>
    `S`: DataFrame with summing matrix of size `(base, bottom)`, see [aggregate function](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
    `tags`: np.ndarray, with hierarchical aggregation indexes, where 
        each key is a level and its value contains tags associated to that level.<br>
    `S_id_col` : str='unique_id', column that identifies each aggregation.<br>

    """
    def __init__(self,
                 S: Frame,
                 tags: dict[str, np.ndarray],
                 S_id_col: str = "unique_id",
                 ):

        self.S = nw.from_native(S)
        S_cols_ex_id_col = self.S.columns
        S_cols_ex_id_col.remove(S_id_col)
        self.S_cols_ex_id_col = S_cols_ex_id_col
        self.tags = tags

    def plot_summing_matrix(self):
        """ Summation Constraints plot
        
        This method simply plots the hierarchical aggregation
        constraints matrix $\mathbf{S}$.
        """
        plt.figure(num=1, figsize=(4, 6), dpi=80, facecolor='w')
        plt.spy(self.S[self.S_cols_ex_id_col].to_numpy())
        plt.show()
        plt.close()

    def plot_series(self,
                    series: str,
                    Y_df: Frame,
                    models: Optional[list[str]] = None,
                    level: Optional[list[int]] = None,
                    id_col: str = "unique_id",
                    time_col: str = "ds",
                    target_col: str = "y",
                    ):
        """ Single Series plot

        **Parameters:**<br>
        `series`: str, string identifying the `'unique_id'` any-level series to plot.<br>
        `Y_df`: DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns `['unique_id', 'ds', 'y']`, it may have `'models'`.<br>
        `models`: list[str], string identifying filtering model columns.<br>
        `level`: float list 0-100, confidence levels for prediction intervals available in `Y_df`.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>           

        **Returns:**<br>
        Single series plot with filtered models and prediction interval level.<br><br>
        """
        Y_nw = nw.from_native(Y_df)

        if series not in self.S[id_col]:
            raise Exception(f'time series {series} not found')
        fig, ax = plt.subplots(1, 1, figsize = (20, 7))
        df_plot = Y_nw.filter(nw.col(id_col) == series)
        cols = models if models is not None else [col for col in df_plot.columns if col not in [id_col, time_col]]
        cols_wo_levels = [col for col in cols if ('-lo-' not in col and '-hi-' not in col)]
        try:
            cmap = plt.get_cmap("tab10", 10)
        except AttributeError:
            cmap = plt.cm.get_cmap("tab10", 10)
        cmap = [cmap(i) for i in range(10)][:len(cols_wo_levels)]
        cmap_dict = dict(zip(cols_wo_levels, cmap))
        for col in cols_wo_levels:
            ax.plot(df_plot[time_col].to_numpy(), df_plot[col].to_numpy(), linewidth=2, label=col, color=cmap_dict[col])
            if level is not None and col != target_col:
                for lv in level:
                    if f'{col}-lo-{lv}' not in df_plot.columns:
                        # if model
                        # doesnt have levels
                        continue
                    ax.fill_between(
                        df_plot.select(nw.col(time_col))[time_col].to_numpy(), 
                        df_plot.select(nw.col(f'{col}-lo-{lv}'))[f'{col}-lo-{lv}'].to_numpy(), 
                        df_plot.select(nw.col(f'{col}-hi-{lv}'))[f'{col}-hi-{lv}'].to_numpy(),
                        alpha=-lv/100 + 1,
                        color=cmap_dict[col],
                        label=f'{col}_level_{lv}'
                    )
        ax.set_title(f'{series} Forecast', fontsize=22)
        ax.set_xlabel('Timestamp [t]', fontsize=20)
        ax.legend(prop={'size': 15})
        ax.grid()
        ax.xaxis.set_major_locator(
            plt.MaxNLocator(min(max(len(df_plot) // 10, 1), 10))
        )
        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(20)
                    
    def plot_hierarchically_linked_series(self,
                                          bottom_series: str,
                                          Y_df: Frame,
                                          models: Optional[list[str]] = None,
                                          level: Optional[list[int]] = None,
                                          id_col: str = "unique_id",
                                          time_col: str = "ds",
                                          target_col: str = "y",                                                              
                                          ):
        """ Hierarchically Linked Series plot

        **Parameters:**<br>
        `bottom_series`: str, string identifying the `'unique_id'` bottom-level series to plot.<br>
        `Y_df`: DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns ['unique_id', 'ds', 'y'] and models. <br>
        `models`: list[str], string identifying filtering model columns.<br>
        `level`: float list 0-100, confidence levels for prediction intervals available in `Y_df`.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>          

        **Returns:**<br>
        Collection of hierarchilly linked series plots associated with the `bottom_series`
        and filtered models and prediction interval level.<br><br>
        """
        Y_nw = nw.from_native(Y_df)

        if bottom_series not in self.S.columns:
            raise Exception(f'bottom time series {bottom_series} not found')

        linked_series = self.S[[id_col, bottom_series]].filter(nw.col(bottom_series) == 1)[id_col].to_numpy()
        fig, axs = plt.subplots(len(linked_series), 1, figsize=(20, 2 * len(linked_series)))
        cols = models if models is not None else [col for col in Y_nw.columns if col not in [id_col, time_col]]
        cols_wo_levels = [col for col in cols if ('-lo-' not in col and '-hi-' not in col)]
        cmap = plt.cm.get_cmap("tab10", 10)
        cmap = [cmap(i) for i in range(10)][:len(cols_wo_levels)]
        cmap_dict = dict(zip(cols_wo_levels, cmap))
        for idx, series in enumerate(linked_series):
            df_plot = Y_nw.filter(nw.col(id_col) == series)
            for col in cols_wo_levels:
                axs[idx].plot(df_plot[time_col].to_numpy(), df_plot[col].to_numpy(), linewidth=2, label=col, color=cmap_dict[col])
                if level is not None and col != target_col:
                    for lv in level:
                        if f'{col}-lo-{lv}' not in df_plot.columns:
                            # if model
                            # doesnt have levels
                            continue
                        axs[idx].fill_between(
                            df_plot.select(nw.col(time_col))[time_col].to_numpy(), 
                            df_plot.select(nw.col(f'{col}-lo-{lv}'))[f'{col}-lo-{lv}'].to_numpy(), 
                            df_plot.select(nw.col(f'{col}-hi-{lv}'))[f'{col}-hi-{lv}'].to_numpy(),                          
                            alpha=-lv/100 + 1,
                            color=cmap_dict[col],
                            label=f'{col}_level_{lv}'
                        )
            axs[idx].set_title(f'{series}', fontsize=10)
            axs[idx].grid()
            axs[idx].get_xaxis().label.set_visible(False)
            axs[idx].legend().set_visible(False)
            axs[idx].xaxis.set_major_locator(
                plt.MaxNLocator(min(max(len(df_plot) // 10, 1), 10))
            )
            for label in (axs[idx].get_xticklabels() + axs[idx].get_yticklabels()):
                label.set_fontsize(10)
        plt.subplots_adjust(hspace=0.4)
        handles, labels = axs[0].get_legend_handles_labels()
        kwargs = dict(loc='lower center', 
                      prop={'size': 10}, 
                      bbox_to_anchor=(0, 0.05, 1, 1))
        if sys.version_info.minor > 7:
            kwargs['ncols'] = np.max([2, np.ceil(len(labels) / 2)])
        fig.legend(handles, labels, **kwargs)

    def plot_hierarchical_predictions_gap(self,
                                          Y_df: Frame,
                                          models: Optional[list[str]] = None,
                                          xlabel: Optional[str] = None,
                                          ylabel: Optional[str] = None,
                                          id_col: str = "unique_id",
                                          time_col: str = "ds",
                                          target_col: str = "y",                                             
                                          ):
        """ Hierarchically Predictions Gap plot

        **Parameters:**<br>
        `Y_df`: DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns ['unique_id', 'ds', 'y'] and models. <br>
        `models`: list[str], string identifying filtering model columns. <br>
        `xlabel`: str, string for the plot's x axis label.<br>
        `ylabel`: str, string for the plot's y axis label.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>              

        **Returns:**<br>
        Plots of aggregated predictions at different levels of the hierarchical structure.
        The aggregation is performed according to the tag levels see 
        [aggregate function](https://nixtla.github.io/hierarchicalforecast/utils.html).<br><br>
        """
        Y_nw = nw.from_native(Y_df)
        
        # Parse predictions dataframe
        horizon_dates = Y_nw['ds'].unique().to_numpy()
        cols = models if models is not None else [col for col in Y_nw.columns if col not in [id_col, time_col]]        
        
        # Plot predictions across tag levels
        fig, ax = plt.subplots(figsize=(8, 5))
        
        if target_col in Y_nw.columns:
            idx_top = self.S.with_columns(sum_cols = nw.sum_horizontal(cols)).sort(by="sum_cols", descending=True)[0][id_col].to_numpy()
            y_plot = Y_nw.filter(nw.col(id_col) == idx_top)[target_col].to_numpy()
            plt.plot(horizon_dates, y_plot, label='True')

        ys = []
        for tag in self.tags:
            y_plot = sum([Y_nw.filter(nw.col(id_col) == idx)[cols].to_numpy() \
                          for idx in self.tags[tag]])
            plt.plot(horizon_dates, y_plot, label=f'Level: {tag}')
            
            ys.append(y_plot[:,None])

        plt.title('Predictions Accumulated Difference')
        if ylabel is not None:
            plt.ylabel(ylabel)
        if xlabel is not None:
            plt.xlabel(xlabel)

        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
show_doc(HierarchicalPlot, title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_summing_matrix, 
         name='plot_summing_matrix', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_series, 
         name='plot_series', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_hierarchically_linked_series, 
         name='plot_hierarchically_linked_series', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_hierarchical_predictions_gap,
         name='plot_hierarchical_predictions_gap', title_level=3)

In [ ]:
#| hide
hplots = HierarchicalPlot(S=S_df, tags=tags)
hplots.plot_summing_matrix()

In [ ]:
#| hide
# polars
S_df_f = pl.from_pandas(S_df.reset_index())
hplots_f = HierarchicalPlot(S=S_df_f, tags=tags)
hplots_f.plot_summing_matrix()

In [ ]:
#| hide
hier_df['Model'] = hier_df['y'] * 1.1
hier_df['Model-lo-80'] = hier_df['Model'] - 0.1 * hier_df['Model']
hier_df['Model-hi-80'] = hier_df['Model'] + 0.1 * hier_df['Model']
hier_df['Model-lo-90'] = hier_df['Model'] - 0.2 * hier_df['Model']
hier_df['Model-hi-90'] = hier_df['Model'] + 0.2 * hier_df['Model']
hplots.plot_series(
    series='Australia', 
    Y_df=hier_df,
    level=[80, 90]
)

In [ ]:
#| hide
# polars
hier_df_f = pl.from_pandas(hier_df)

hplots_f.plot_series(
    series='Australia', 
    Y_df=hier_df_f,
    level=[80, 90]
)

In [ ]:
#| hide
hplots.plot_series(series='Australia', 
                   Y_df=hier_df)

In [ ]:
#| hide
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df,
    level=[80, 90]
)

In [ ]:
#| hide
# polars
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df_f,
    level=[80, 90]
)

In [ ]:
#| hide
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df,
)

In [ ]:
#| hide
# test series with just one value
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df.groupby('unique_id').tail(1),
)

In [ ]:
#| hide
# polars
hplots_f.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=pl.from_pandas(hier_df.groupby('unique_id').tail(1)),
)

In [ ]:
#| hide
hplots.plot_hierarchical_predictions_gap(Y_df=hier_df.drop(columns='y'), models=['Model'])

In [ ]:
#| hide
# polars
hplots_f.plot_hierarchical_predictions_gap(Y_df=pl.from_pandas(hier_df.drop(columns='y')), models=['Model'])

In [ ]:
#| eval: false
from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS
from datasetsforecast.hierarchical import HierarchicalData

Y_df, S, tags = HierarchicalData.load('./data', 'Labour')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
S = S.reset_index(names="unique_id")

Y_test_df  = Y_df.groupby('unique_id').tail(24)
Y_train_df = Y_df.drop(Y_test_df.index)

fcst = StatsForecast( 
    models=[AutoETS(season_length=12, model='AAZ')],
    freq='MS', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(df=Y_train_df, h=24).reset_index()

# Plot prediction difference of different aggregation
# Levels Country, Country/Region, Country/Gender/Region ...
hplots = HierarchicalPlot(S=S, tags=tags)

hplots.plot_hierarchical_predictions_gap(
    Y_df=Y_hat_df, models='AutoETS',
    xlabel='Month', ylabel='Predictions',
)

In [ ]:
#| eval: false
# polars
from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS
from datasetsforecast.hierarchical import HierarchicalData

Y_df, S, tags = HierarchicalData.load('./data', 'Labour')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
S = S.reset_index(names="unique_id")

Y_test_df  = Y_df.groupby('unique_id').tail(24)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df_pl  = pl.from_pandas(Y_test_df)
Y_train_df_pl = pl.from_pandas(Y_train_df)

fcst = StatsForecast(
    models=[AutoETS(season_length=12, model='AAZ')],
    freq='1m', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(df=Y_train_df_pl, h=24)

# Plot prediction difference of different aggregation
# Levels Country, Country/Region, Country/Gender/Region ...
hplots = HierarchicalPlot(S=S, tags=tags)

hplots.plot_hierarchical_predictions_gap(
    Y_df=Y_hat_df, models='AutoETS',
    xlabel='Month', ylabel='Predictions',
)

# External Forecast Adapters

In [ ]:
#| exporti

# convert levels to output quantile names
def level_to_outputs(level: list[int]) -> tuple[list[float], list[str]]:
    """ Converts list of levels into output names matching StatsForecast and NeuralForecast methods.

    **Parameters:**<br>
    `level`: int list [0,100]. Probability levels for prediction intervals.<br>

    **Returns:**<br>
    `output_names`: str list. String list with output column names.
    """
    qs = sum([[50-l/2, 50+l/2] for l in level], [])
    output_names = sum([[f'-lo-{l}', f'-hi-{l}'] for l in level], [])

    sort_idx = np.argsort(qs)
    quantiles = np.array(qs)[sort_idx]

    # Add default median
    quantiles = np.concatenate([np.array([50]), quantiles]) / 100
    output_names = list(np.array(output_names)[sort_idx])
    output_names.insert(0, '-median')
    
    return quantiles, output_names

# convert quantiles to output quantile names
def quantiles_to_outputs(quantiles: list[float]) -> tuple[list[float], list[str]]:
    """Converts list of quantiles into output names matching StatsForecast and NeuralForecast methods.

    **Parameters:**<br>
    `quantiles`: float list [0., 1.]. Alternative to level, quantiles to estimate from y distribution.<br>

    **Returns:**<br>
    `output_names`: str list. String list with output column names.
    """
    output_names = []
    for q in quantiles:
        if q<.50:
            output_names.append(f'-lo-{np.round(100-200*q,2)}')
        elif q>.50:
            output_names.append(f'-hi-{np.round(100-200*(1-q),2)}')
        else:
            output_names.append('-median')
    return quantiles, output_names

In [ ]:
#| exporti

# given input array of sample forecasts and inptut quantiles/levels, 
# output a Pandas Dataframe with columns of quantile predictions
def samples_to_quantiles_df(samples: np.ndarray, 
                            unique_ids: Sequence[str], 
                            dates: list[str], 
                            quantiles: Optional[list[float]] = None,
                            level: Optional[list[int]] = None, 
                            model_name: str = "model",
                            id_col: str = 'unique_id',
                            time_col: str = 'ds',
                            backend: str = 'pandas',
                            ) -> tuple[list[float], FrameT]:
    """ Transform Random Samples into HierarchicalForecast input.
    Auxiliary function to create compatible HierarchicalForecast input `Y_hat_df` dataframe.

    **Parameters:**<br>
    `samples`: numpy array. Samples from forecast distribution of shape [n_series, n_samples, horizon].<br>
    `unique_ids`: string list. Unique identifiers for each time series.<br>
    `dates`: datetime list. list of forecast dates.<br>
    `quantiles`: float list in [0., 1.]. Alternative to level, quantiles to estimate from y distribution.<br>
    `level`: int list in [0,100]. Probability levels for prediction intervals.<br>
    `model_name`: string. Name of forecasting model.<br>
    `id_col` : str='unique_id', column that identifies each serie.<br>
    `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
    `backend` : str='pandas', backend to use for the output dataframe, either 'pandas' or 'polars'.<br>

    **Returns:**<br>
    `quantiles`: float list in [0., 1.]. quantiles to estimate from y distribution .<br>
    `Y_hat_df`: DataFrame. With base quantile forecasts with columns ds and models to reconcile indexed by unique_id.
    """
    
    # Get the shape of the array
    n_series, n_samples, horizon = samples.shape

    if n_series != len(unique_ids):
        raise ValueError(
            f"Number of unique_ids ({len(unique_ids)}) must match the number of series ({n_series})."
        )
    if horizon != len(dates):
        raise ValueError(
            f"Number of dates ({len(dates)}) must match third dimension of samples array ({horizon})."
        )
    if not ((quantiles is None) ^ (level is None)):
        raise ValueError("Either quantiles or level must be provided, but not both.")

    namespace = sys.modules.get(backend, None)
    if namespace is None:
        raise ValueError(f"DataFrame backend {backend} not installed.")

    #create initial dictionary
    forecasts_mean = np.mean(samples, axis=1).flatten()
    unique_ids = np.repeat(unique_ids, horizon)
    ds = np.tile(dates, n_series)

    #create quantiles and quantile names
    if level is not None:
        _quantiles, quantile_names = level_to_outputs(level)
    elif quantiles is not None:
        _quantiles, quantile_names = quantiles_to_outputs(quantiles)

    percentiles = [quantile * 100 for quantile in _quantiles]
    col_names = np.array([model_name + quantile_name for quantile_name in quantile_names])
    
    #add quantiles to dataframe
    forecasts_quantiles = np.percentile(samples, percentiles, axis=1)

    forecasts_quantiles = np.transpose(forecasts_quantiles, (1,2,0)) # [Q,H,N] -> [N,H,Q]
    forecasts_quantiles = forecasts_quantiles.reshape(-1,len(_quantiles))
    
    df_nw = nw.from_dict(
        {
            **{id_col: unique_ids, time_col: ds, model_name: forecasts_mean},
            **dict(zip(col_names, forecasts_quantiles.T)),
        },
        native_namespace=namespace,
    )

    return _quantiles, df_nw.to_native()

In [ ]:
show_doc(samples_to_quantiles_df, title_level=3)

In [ ]:
#| hide

#level_to_outputs unit tests
test_eq(
    level_to_outputs([80, 90]),
    ([0.5 , 0.05, 0.1 , 0.9 , 0.95], ['-median', '-lo-90', '-lo-80', '-hi-80', '-hi-90'])
)
test_eq(
    level_to_outputs([30]),
    ([0.5 , 0.35, 0.65], ['-median', '-lo-30', '-hi-30'])
)
#quantiles_to_outputs unit tests
test_eq(
    quantiles_to_outputs([0.2, 0.4, 0.6, 0.8]),
    ([0.2,0.4,0.6, 0.8], ['-lo-60.0', '-lo-20.0', '-hi-20.0', '-hi-60.0'])
)
test_eq(
    quantiles_to_outputs([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
    ([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
     ['-lo-80.0', '-lo-60.0', '-lo-40.0', '-lo-20.0', '-median', '-hi-20.0', '-hi-40.0', '-hi-60.0', '-hi-80.0'])
)

In [ ]:
#| hide

#samples_to_quantiles_df unit tests
start_date = pd.Timestamp("2023-06-01")
end_date = pd.Timestamp("2023-06-10")
frequency = "D"  # Daily frequency
dates = pd.date_range(start=start_date, end=end_date, freq=frequency).tolist()
samples = np.random.rand(3, 200, 10)
unique_ids = ['id1', 'id2', 'id3']
level = np.array([10, 50, 90])
quantiles=np.array([0.5, 0.05, 0.25, 0.45, 0.55, 0.75, 0.95])

ret_quantiles_1, ret_df_1 = samples_to_quantiles_df(samples, unique_ids, dates, level=level)
ret_quantiles_2, ret_df_2 = samples_to_quantiles_df(samples, unique_ids, dates, quantiles=quantiles)

test_eq(
    ret_quantiles_1,
    quantiles
)
test_eq(
    ret_df_1.columns,
    ['unique_id', 'ds', 'model', 'model-median', 'model-lo-90', 'model-lo-50', 'model-lo-10', 'model-hi-10', 'model-hi-50', 'model-hi-90']
)
test_eq(
    ret_df_1["unique_id"].values,
    ['id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1',
       'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2',
       'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3']
)
test_eq(
    ret_quantiles_1, ret_quantiles_2
)
test_eq(
    ret_df_1["unique_id"], ret_df_2["unique_id"]
)

In [ ]:
#| hide
# polars

ret_quantiles_1, ret_df_1 = samples_to_quantiles_df(samples, unique_ids, dates, level=level, backend='polars')
ret_quantiles_2, ret_df_2 = samples_to_quantiles_df(samples, unique_ids, dates, quantiles=quantiles, backend='polars')

test_eq(
    ret_quantiles_1,
    quantiles
)
test_eq(
    ret_df_1.columns,
    ['unique_id', 'ds', 'model', 'model-median', 'model-lo-90', 'model-lo-50', 'model-lo-10', 'model-hi-10', 'model-hi-50', 'model-hi-90']
)
test_eq(
    list(ret_df_1["unique_id"]),
    ['id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1', 'id1',
       'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2', 'id2',
       'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3']
)
test_eq(
    ret_quantiles_1, ret_quantiles_2
)
test_eq(
    ret_df_1["unique_id"], ret_df_2["unique_id"]
)

In [ ]:
#| hide

#Unit test for the aggregate function accounting for exog_vars

df = pd.DataFrame(
    {
        'cat1': ['a', 'a', 'c'],
        'cat2': ['1', '2', '3'],
        'exog1': [4, 5, 6],
        'exog2': [7, 6, 5],
        'y': [10, 20, 30],
        'ds': ['2020-01-01', '2020-02-01', '2020-03-01']
    }
)
spec = [["cat1"],["cat1","cat2"]]


Y_df_check = pd.DataFrame(
    data={
        'unique_id': ['a', 'a', 'c', 'a/1', 'a/2', 'c/3'],
        'ds': ['2020-01-01','2020-02-01','2020-03-01','2020-01-01','2020-02-01','2020-03-01'],
        'y': [10, 20, 30, 10, 20, 30],
    },
)

S_df_check = pd.DataFrame(
    data={
        'unique_id': ['a','c','a/1','a/2','c/3'],
        'a/1': np.array([1.0, 0.0, 1.0, 0.0, 0.0], dtype=np.float64),
        'a/2': np.array([1.0, 0.0, 0.0, 1.0, 0.0], dtype=np.float64),
        'c/3': np.array([0.0, 1.0, 0.0, 0.0, 1.0], dtype=np.float64)
    },
)


Y_df_check_exog = pd.DataFrame(
    data = {
        'unique_id': ['a', 'a', 'c', 'a/1', 'a/2', 'c/3'],
        'ds': ['2020-01-01', '2020-02-01', '2020-03-01', '2020-01-01', '2020-02-01', '2020-03-01'],
        'y': [10, 20, 30, 10, 20, 30],
        'exog1_mean': [4.0, 5.0, 6.0, 4.0, 5.0, 6.0],
        'exog2_sum': [7, 6, 5, 7, 6, 5]
    },
)

Y_df, S_df, tags = aggregate(
    df = df,
    spec = spec,
    exog_vars = None,
)

Y_df_exog, S_df_exog, tags = aggregate(
    df = df,
    spec = spec,
    exog_vars = {'exog1':'mean','exog2':'sum'},
)

test_eq(Y_df, 
        Y_df_check)

test_eq(S_df, 
        S_df_check)

test_eq(Y_df_exog,
        Y_df_check_exog)

In [ ]:
#| hide
# polars

df_f = pl.from_pandas(df)
Y_df_check_f = pl.from_pandas(Y_df_check)
S_df_check_f = pl.from_pandas(S_df_check)
Y_df_check_exog_f = pl.from_pandas(Y_df_check_exog)

Y_df, S_df, tags = aggregate(
    df = df_f,
    spec = spec,
    exog_vars = None,
)

Y_df_exog, S_df_exog, tags = aggregate(
    df = df_f,
    spec = spec,
    exog_vars = {'exog1':'mean','exog2':'sum'},
)

test_eq(Y_df, 
        Y_df_check_f)

test_eq(S_df, 
        S_df_check_f)

test_eq(Y_df_exog,
        Y_df_check_exog_f)

In [ ]:
#| exporti

# Masked empirical covariance matrix
@njit("Array(float64, 2, 'F')(Array(float64, 2, 'C'), Array(bool_, 2, 'C'))", nogil=NUMBA_NOGIL, cache=NUMBA_CACHE, parallel=NUMBA_PARALLEL, fastmath=NUMBA_FASTMATH, error_model="numpy")
def _ma_cov(residuals: np.ndarray, not_nan_mask: np.ndarray):
    """Masked empirical covariance matrix.

    :meta private:
    """
    n_timeseries = residuals.shape[0]
    W = np.zeros((n_timeseries, n_timeseries), dtype=np.float64).T
    for i in prange(n_timeseries):
        not_nan_mask_i = not_nan_mask[i]
        for j in range(i + 1):
            not_nan_mask_j = not_nan_mask[j]
            not_nan_mask_ij = not_nan_mask_i & not_nan_mask_j   
            n_samples = np.sum(not_nan_mask_ij)
            # Only compute if we have enough non-nan samples in the time series pair
            if n_samples > 1:
                # Masked residuals
                residuals_i = residuals[i][not_nan_mask_ij]
                residuals_j = residuals[j][not_nan_mask_ij]
                residuals_i_mean = np.mean(residuals_i)
                residuals_j_mean = np.mean(residuals_j)
                X_i = (residuals_i - residuals_i_mean)
                X_j = (residuals_j - residuals_j_mean)
                # Empirical covariance
                factor_emp_cov = np.float64(1 / (n_samples - 1))
                W[i, j] = W[j, i] = factor_emp_cov * np.sum(X_i * X_j)

    return W

In [ ]:
#| exporti

# Shrunk covariance matrix using the Schafer-Strimmer method

@njit("Array(float64, 2, 'F')(Array(float64, 2, 'C'), float64)", nogil=NUMBA_NOGIL, cache=NUMBA_CACHE, parallel=NUMBA_PARALLEL, fastmath=NUMBA_FASTMATH, error_model="numpy")
def _shrunk_covariance_schaferstrimmer_no_nans(residuals: np.ndarray, mint_shr_ridge: float):
    """Shrink empirical covariance according to the following method:
        Schäfer, Juliane, and Korbinian Strimmer. 
        ‘A Shrinkage Approach to Large-Scale Covariance Matrix Estimation and 
        Implications for Functional Genomics’. Statistical Applications in 
        Genetics and Molecular Biology 4, no. 1 (14 January 2005). 
        https://doi.org/10.2202/1544-6115.1175.

    :meta private:
    """
    n_timeseries = residuals.shape[0]
    n_samples = residuals.shape[1]
    
    # We need the empirical covariance, the off-diagonal sum of the variance of 
    # the empirical correlation matrix and the off-diagonal sum of the squared 
    # empirical correlation matrix.
    W = np.zeros((n_timeseries, n_timeseries), dtype=np.float64).T
    sum_var_emp_corr = np.float64(0.0)
    sum_sq_emp_corr = np.float64(0.0)
    factor_emp_cov = np.float64(1 / (n_samples - 1))
    factor_shrinkage = np.float64(1 / (n_samples * (n_samples - 1)))
    epsilon = np.float64(2e-8)
    for i in prange(n_timeseries):
        # Mean of the standardized residuals
        X_i = residuals[i] - np.mean(residuals[i])
        Xs_i = X_i / (np.std(residuals[i]) + epsilon)
        Xs_i_mean = np.mean(Xs_i)
        for j in range(i + 1):
            # Empirical covariance
            X_j = residuals[j] - np.mean(residuals[j])
            W[i, j] = factor_emp_cov * np.sum(X_i * X_j)
            # Off-diagonal sums
            if i != j:
                Xs_j = X_j / (np.std(residuals[j]) + epsilon)
                Xs_j_mean = np.mean(Xs_j)
                # Sum off-diagonal variance of empirical correlation
                w = (Xs_i - Xs_i_mean) * (Xs_j - Xs_j_mean)
                w_mean = np.mean(w)
                sum_var_emp_corr += np.sum(np.square(w - w_mean))
                # Sum squared empirical correlation
                sum_sq_emp_corr += w_mean**2

    # Calculate shrinkage intensity 
    shrinkage = 1.0 - max(min((factor_shrinkage * sum_var_emp_corr) / (sum_sq_emp_corr + epsilon), 1.0), 0.0)
    # Shrink the empirical covariance
    for i in prange(n_timeseries):
        for j in range(i + 1):
            if i != j:    
                W[i, j] = W[j, i] = shrinkage * W[i, j]
            else:
                W[i, j] = W[j, i] = max(W[i, j], mint_shr_ridge)
    return W

@njit("Array(float64, 2, 'F')(Array(float64, 2, 'C'), Array(bool_, 2, 'C'), float64)", nogil=NUMBA_NOGIL, cache=NUMBA_CACHE, parallel=NUMBA_PARALLEL, fastmath=NUMBA_FASTMATH, error_model="numpy")
def _shrunk_covariance_schaferstrimmer_with_nans(residuals: np.ndarray, not_nan_mask: np.ndarray, mint_shr_ridge: float):
    """Shrink empirical covariance according to the following method:
        Schäfer, Juliane, and Korbinian Strimmer. 
        ‘A Shrinkage Approach to Large-Scale Covariance Matrix Estimation and 
        Implications for Functional Genomics’. Statistical Applications in 
        Genetics and Molecular Biology 4, no. 1 (14 January 2005). 
        https://doi.org/10.2202/1544-6115.1175.

    :meta private:
    """
    n_timeseries = residuals.shape[0]
    
    # We need the empirical covariance, the off-diagonal sum of the variance of 
    # the empirical correlation matrix and the off-diagonal sum of the squared 
    # empirical correlation matrix.
    W = np.zeros((n_timeseries, n_timeseries), dtype=np.float64).T
    sum_var_emp_corr = np.float64(0.0)
    sum_sq_emp_corr = np.float64(0.0)
    epsilon = np.float64(2e-8)
    for i in prange(n_timeseries):
        not_nan_mask_i = not_nan_mask[i]
        for j in range(i + 1):
            not_nan_mask_j = not_nan_mask[j]
            not_nan_mask_ij = not_nan_mask_i & not_nan_mask_j   
            n_samples = np.sum(not_nan_mask_ij)
            # Only compute if we have enough non-nan samples in the time series pair
            if n_samples > 1:
                # Masked residuals
                residuals_i = residuals[i][not_nan_mask_ij]
                residuals_j = residuals[j][not_nan_mask_ij]
                residuals_i_mean = np.mean(residuals_i)
                residuals_j_mean = np.mean(residuals_j)
                X_i = (residuals_i - residuals_i_mean)
                X_j = (residuals_j - residuals_j_mean)
                # Empirical covariance
                factor_emp_cov = np.float64(1 / (n_samples - 1))
                W[i, j] = factor_emp_cov * np.sum(X_i * X_j)
                # Off-diagonal sums
                if i != j:
                    factor_var_emp_cor = np.float64(n_samples / (n_samples - 1)**3)
                    residuals_i_std = np.std(residuals_i) + epsilon
                    residuals_j_std = np.std(residuals_j) + epsilon
                    Xs_i = X_i / (residuals_i_std + epsilon)
                    Xs_j = X_j / (residuals_j_std + epsilon)
                    Xs_im_mean = np.mean(Xs_i)
                    Xs_jm_mean = np.mean(Xs_j)
                    # Sum off-diagonal variance of empirical correlation
                    w = (Xs_i - Xs_im_mean) * (Xs_j - Xs_jm_mean)
                    w_mean = np.mean(w)
                    sum_var_emp_corr += factor_var_emp_cor * np.sum(np.square(w - w_mean))
                    # Sum squared empirical correlation
                    sum_sq_emp_corr += np.square(factor_emp_cov * n_samples * w_mean)

    # Calculate shrinkage intensity 
    shrinkage = 1.0 - max(min((sum_var_emp_corr) / (sum_sq_emp_corr + epsilon), 1.0), 0.0)

    # Shrink the empirical covariance
    for i in prange(n_timeseries):
        for j in range(i + 1):
            if i != j:    
                W[i, j] = W[j, i] = shrinkage * W[i, j]
            else:
                W[i, j] = W[j, i] = max(W[i, j], mint_shr_ridge)

    return W

In [ ]:
#| hide
# test covariance equivalence
n_samples = 100
n_hiers = 10
y_insample = np.random.rand(n_samples, n_hiers)
y_hat_insample = np.random.rand(n_samples, n_hiers)
residuals = (y_insample - y_hat_insample)
nan_mask = np.isnan(residuals)

# Check equivalence of covariance functions in case of no nans
W_nb = _ma_cov(residuals, ~nan_mask)
W_np = np.cov(residuals)
np.testing.assert_allclose(W_nb, W_np, atol=1e-6)

# Check equivalence of shrunk covariance functions in case of no nans
W_ss_nonan = _shrunk_covariance_schaferstrimmer_no_nans(residuals, 2e-8)
W_ss_nan = _shrunk_covariance_schaferstrimmer_with_nans(residuals, ~nan_mask, 2e-8)
np.testing.assert_allclose(W_ss_nan, W_ss_nonan, atol=1e-6)

# Check equivalence of diagonal elements of shrunk W to non-shrunk W in case of no nans
np.testing.assert_allclose(np.diag(W_np), np.diag(W_ss_nan), atol=1e-6)

In [ ]:
#| exporti

# Lasso cyclic coordinate descent
@njit("Array(float64, 1, 'C')(Array(float64, 2, 'C'), Array(float64, 1, 'C'), float64, int64, float64)", nogil=NUMBA_NOGIL, cache=NUMBA_CACHE, fastmath=NUMBA_FASTMATH, error_model="numpy")
def _lasso(X: np.ndarray, y: np.ndarray, 
          lambda_reg: float, max_iters: int = 1_000,
          tol: float = 1e-4):
    # lasso cyclic coordinate descent
    n, feats = X.shape
    norms = np.sum(X ** 2, axis=0)
    beta = np.zeros(feats, dtype=np.float64)
    beta_changes = np.zeros(feats, dtype=np.float64)
    residuals = y.copy()

    for it in range(max_iters):
        for i in range(feats):            
            norms_i = norms[i]
            # is feature is close to zero, we 
            # continue to the next.
            # in this case is optimal betai= 0
            if abs(norms_i) < 1e-8:
                continue
            beta_i = beta[i]

            #we calculate the normalized derivative
            rho = beta_i
            for j in range(n):
                rho += X[j, i] * residuals[j] / norms_i

            #soft threshold
            beta_i_next = np.sign(rho) * max(np.abs(rho) - lambda_reg * n / norms_i, 0.)#(norms[i] + 1e-3), 0.)
            beta_delta = beta_i - beta_i_next
            beta_changes[i] = np.abs(beta_delta)
            if beta_delta != 0.0:
                for j in range(n):
                    residuals[j] += beta_delta * X[j, i]

                beta[i] = beta_i_next
        
        if max(beta_changes) < tol:
            break

    return beta